In [1]:
import numpy as np
import cv2
from scipy import ndimage
from PIL import Image

In [6]:
import numpy as np

n = 10 # количество ядер свертки
window_size = 3 # размер окна свертки

kernels = np.zeros((n, window_size, window_size))

for i in range(n):
    kernels[i] = np.random.randn(window_size, window_size)
    kernels[i] /= sum(kernels[i])
    
print(kernels)


[[[ 2.30186191e+00  6.40050321e-01  5.94788264e-01]
  [-1.16105837e+00 -1.30626837e-01  3.54059770e-01]
  [-1.40803546e-01  4.90576516e-01  5.11519667e-02]]

 [[-3.80921218e-01 -2.61311524e-01 -2.46846182e+01]
  [ 2.44514088e-01  6.86844020e-01  4.99400224e+01]
  [ 1.13640713e+00  5.74467504e-01 -2.42554042e+01]]

 [[ 1.28689223e+00 -9.98003209e-03  7.35666998e-01]
  [ 2.35557065e+00  6.73133390e-01  1.76311616e-01]
  [-2.64246287e+00  3.36846642e-01  8.80213856e-02]]

 [[ 1.03098225e+00 -2.16229206e-01  7.12689501e-01]
  [ 1.93422672e-01  3.01120097e-01 -9.11793619e-01]
  [-2.24404923e-01  9.15109109e-01  1.19910412e+00]]

 [[ 6.47252584e-01  1.22168913e+00 -4.80811492e-01]
  [ 5.18205119e-01  1.98585017e-01 -9.60671586e-01]
  [-1.65457703e-01 -4.20274143e-01  2.44148308e+00]]

 [[ 2.76222963e-01  6.73269027e-01  1.30784261e+00]
  [ 5.59452808e-01 -6.35044532e-02 -6.14124668e-01]
  [ 1.64324229e-01  3.90235426e-01  3.06282059e-01]]

 [[ 6.26245132e-01 -1.96563395e+01  5.15458604e-01]


In [7]:
img_src = np.array(Image.open('../data/data3/cats/0.jpg').resize((400, 200)), dtype=np.float64)

In [8]:
img_src /= 255

In [7]:
np.sum(cv2.filter2D(img_src, -1, kernels[0]), axis=2)

array([[ 5.34117647,  5.36470588,  5.41176471, ...,  6.64242958,
         6.10901643,  6.10265646],
       [ 5.34117647,  5.36470588,  5.41176471, ...,  6.66994796,
         7.10035281,  7.13856688],
       [ 5.34117647,  5.36470588,  5.41176471, ...,  7.20623999,
         6.65243095,  7.12671878],
       ...,
       [ 3.07597734,  3.29615043,  3.24560485, ...,  0.0676379 ,
         0.0676379 ,  0.0676379 ],
       [ 3.40266667,  3.36720169,  2.88768406, ..., -0.45128038,
        -0.45128038, -0.45128038],
       [ 3.36437547,  2.87300566,  3.37614018, ..., -0.29234668,
        -0.29234668, -0.29234668]])

In [8]:
cv2.imshow('res', cv2.filter2D(img_src, -1, np.array([[0.0, -1.0, 0.0], [-1.0, 4.0, -1.0], [0.0, -1.0, 0.0]])))

cv2.waitKey(0)

cv2.destroyAllWindows()

In [9]:
filterred = np.zeros((n, img_src.shape[0], img_src.shape[1]))

In [10]:
for num, kernel in enumerate(kernels):
    filterred[num] = np.sum(cv2.filter2D(img_src, -1, kernel), axis=2)

In [11]:
filterred.shape

(10, 200, 400)

In [12]:
b = np.zeros((200, 400))

In [13]:
filterred += b

In [13]:
def maxpooling(img, window_size, stride):
    """
    img - двумерный массив размером (H, W)
    window_size - длина стороны окна для пулинга
    stride - шаг окна по горизонтали и вертикали
    """
    H, W = img.shape

    # Вычисляем размеры выходного изображения после max-пулинга
    out_height = (H - window_size) // stride + 1
    out_width = (W - window_size) // stride + 1

    # Инициализируем массив для выходного изображения, заполняя его нулями
    output = np.zeros((out_height, out_width), dtype=img.dtype)

    # Проходимся по каждому окну и находим максимальный элемент
    for i in range(out_height):
        for j in range(out_width):
            row_start = i * stride
            row_end = row_start + window_size
            col_start = j * stride
            col_end = col_start + window_size
            window = img[row_start:row_end, col_start:col_end]
            output[i, j] = np.max(window)

    return output


In [14]:
import numpy as np

def averagepooling(image, kernel_size, stride):
    # Размеры изображения
    input_dim_x, input_dim_y = image.shape
    
    # Размеры окна
    kernel_dim_x, kernel_dim_y = kernel_size
    
    # Вычисляем размерность выходного изображения
    output_dim_x = int((input_dim_x - kernel_dim_x) / stride + 1)
    output_dim_y = int((input_dim_y - kernel_dim_y) / stride + 1)
    
    # Создаем выходной массив
    output = np.zeros((output_dim_x, output_dim_y))
    
    # Применяем операцию пулинга к изображению
    for i in range(output_dim_x):
        for j in range(output_dim_y):
            output[i, j] = np.mean(
                image[i * stride : i * stride + kernel_dim_x, j * stride : j * stride + kernel_dim_y]
            )
    
    return output


In [21]:
pooled = []

In [22]:
for num, im in enumerate(filterred):
    # np.concatenate((pooled, averagepooling(im, (3, 3), 2)))
    pooled.append(averagepooling(im, (3, 3), 2))

In [24]:
res1 = np.array(pooled)